> **Jupyter slideshow:** This notebook can be displayed as slides. To view it as a slideshow in your browser type in the console:


> `> jupyter nbconvert [this_notebook.ipynb] --to slides --post serve`


> To toggle off the slideshow cell formatting, click the `CellToolbar` button, then `View --> Cell Toolbar --> None`

<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Flask on AWS

_Author: Phillippa Thomson (NYC) _

---

<a id="learning-objectives"></a>
### Learning Objectives
*After this lesson, you will be able to:*
- Launch an EC2 box with an Apache webserver and Flask app

### Lesson Guide

- [Introduction](#introduction)


- Demo/Guided Practice: Launching an EC2 Box
 - [Step 1 - Log in to EC2](#step--log-in-to-ec)
 - [Step 2 - Launch EC2 Instance](#step----launch-ec-instance)
 - [Step 3 - Select an AMI](#step----select-an-ami)
 - [Step 4 - Select EC2 Instance Type](#step----select-ec-instance-type)
 - [Step 5 - Configure Security Group](#step----configure-security-group)
 - [Step 6 - Connect to your EC2 Instance](#step----connect-to-your-ec-instance)
 - [Step 7 - Let's test the Apache server to see how it works](#step----install-apache)
 - [Step 8 - Import your Flask app](#step----set-up-hello-world-page)
 - [Step 9 - Create WSGI file](#step----run-a-flask-app)
 - [Step 10 - Showtime!](#step----set-up-a-webform)


- [Independent Practice](#independent-practice)
- [Conclusion](#conclusion)
- [Additional Resources](#additional-resources)


<a id="introduction"></a>
## Introduction
---

In the previous lesson, we've learnt the basics of building a Flask application. Now lets assume you have finished building your new, awesome, world changing application. You don’t want to have to tell everyone, “Go check out my awesome new tool @ www.example.com:5000” that would be pretty lame. Not to mention to other drawbacks of using a development environment in production. Here I am going to show how we can use an Apache module called WSGI (Web Server Gateway Interface), to point Apache to your Flask application. We'll do all of this on a public-facing EC2 instance.

By making a website, you can retrieve the latest data from your database and create interactive visuals on-the-fly. Websites are typically much easier for clients and employers to use than Jupyter Notebooks.

Hopefully, you'll use this as inspiration for showcasing your projects!

Here, we'll set up a new EC2 instance that will host our website. We will initialize it using an existing AMI (Amazon Machine Image) that contains Python, pip, Apache and a number of other tools. Otherwise, we would start with just an operating system and would have to install tools manually.

<a id="step--log-in-to-ec"></a>
### Step 1 - Log in to EC2

Log into AWS and select EC2.

<img src="images/ec2-1.png">

<a id="step----launch-ec-instance"></a>
### Step 2 - Launch EC2 Instance

In the EC2 dashboard, click `Launch Instance`.

<img src="images/ec2-2.png">

<a id="step----select-an-ami"></a>
### Step 3 - Select an AMI

On the left, when configuring the box, select 'Community AMI'. Then search for `sg-dsi-01` and select the box called `sg-dsi-01`. Select that and continue on with the defaults.

<img src="images/ec2-4v2.png">

<a id="step----select-ec-instance-type"></a>
### Step 4 - Select EC2 Instance Type

Choose the pre-selected t2-micro box. Then click - at the top `Step 6: Configure Security Group`
    
<img src="images/ec2-4.png">

<a id="step----configure-security-group"></a>
### Step 5 - Configure Security Group

Make sure that in security groups for the box that port 80 (HTTP) and 22 (SSH) are open and reachable from anywhere. Then at the bottom, click `Review and Launch`.

<img src="images/ec2-5.png">

<a id="step----connect-to-your-ec-instance"></a>
### Step 6 - Connect to your EC2 Instance

Return to the EC2 instances screen. You will see the box `pending` as it starts up. Once that is complete, tick the box next to the instance and click `Connect`.

<img src="images/ec2-6.png">

To connect to the AWS instance, use SSH.

- **Windows**: Download [PuTTY](https://winscp.net/eng/download.php) and connect by supplying the DNS name and private key.
- **Mac**: Use the `ssh` command in your terminal.<br>*Example*: `ssh -i cs188.pem ubuntu@ec2-54-169-33-117.ap-southeast-1.compute.amazonaws.com`<br> in this format `ssh -i [KEY_NAME].pem ubuntu@[AWS_DNS_NAME]`


## Skip this step, since it's already installed in the AMI

~~Install Apache webserver. Back at the terminal, run the following commnads:~~

~~```
sudo apt-get update
sudo apt-get install apache2
sudo apt-get install libapache2-mod-wsgi
```~~

#### You can now navigate to your AWS IP address on your browser to see this:

<img src="images/ec2-7.png">

<a id="step----set-up-hello-world-page"></a>
### Step 7 - Let's test the Apache server to see how it works

At the command line, type in the following to create a webpage:

```
cd /var/www/html
sudo mkdir sample
cd sample
sudo touch index.html
sudo chmod 755 index.html
sudo nano index.html
```

What did we do there? We moved into the Apache directory that holds the files that are served as webpages. We then made a new directory there called `sample`. We then changed into that directory and created a page called `index.html`. Finally, we set the permissions on the file and opened it in our text editor.

Type in something like the following: `"Hello from DSI!"`. Once you have that done, save and exit.

Once that is done, we can view our webpage. Go the the address you did earlier for the box in your browser, but this time add `'/sample'` or `'/sample/index.html'` to the end.

If everything went according to plan, you will see the following:

<img src="images/browser-1.png">

Congrats! You have the most boring page on the web! Let's now make it better by adding a flask app back-end.

<a id="step----run-a-flask-app"></a>
### Step 8 - Import your Flask App

Now let's import all files we've modified previously in the `titanic_app` folder into `myapp` folder. To do so, you can upload all files into a GitHub repository, and then use `git clone https://github.com/USERNAME/REPO.git` to clone the Flask application you've saved onto the Github repository. To update the repository, use `git pull`.

In the **sg-dsi-01** AMI image, the filepaths and environment has been setup for you already. Clone your Flask app repository into your home directory first. In your home directory, you should find the folder `flask-apache`. This will be the folder in which your Flask application will reside in. Now, move all files in your cloned repo to `flask-apache`. **Do not nest your folders.** It should look something similar to the following after the above is done:

<img src="images/flask1.PNG">

<a id="step----create-wsgi"></a>
### Step 9 - Create a new WSGI file

Next we are going to create a WSGI file, this is a file which Apache will use to access your Flask application. Python isn’t natively a web language and so this WSGI file allows Apache to interact with Python as if it’s native. It’s a simple script. In the `flask-apache` folder, create `flask.wsgi` with the following:

```bash
touch flask.wsgi
nano flask.wsgi
```

After this, you should get something similar in your directory:

<img src="images/flask2.PNG">

Paste the following into the WSGI file:
    
```python
import sys
sys.path.append('/home/ubuntu/flask-apache')

from controller import app as application
```

**Line 1** is simply importing the system module. **Line 2** we are adding the root path (where the Flask application is located) into the system path. **Line 3** is essentially importing the `app` variable we created in `controller.py` and aliasing it to the application namespace. Apache executes this file after getting a request to our new website and then does it’s magic from there. It is very important that whatever you name the `controller.py` file, you must use the same name in the from area on **line 4** in your WSGI file. `from controller` is really saying `from controller.py`

Once that is done, save and exit. Then, let's tell Apache to enable our new Flask application and disable the old default config and files. To do so, type the following:

```bash
cd ~
sudo a2ensite flask-apache.conf
sudo a2dissite 000-default.conf
sudo /etc/init.d/apache2 restart
```

<a id="step----showtime"></a>
### Step 10 - Showtime!

Now when you go to your webpage 'http://ec2-[IP].[REGION].compute.amazonaws.com/' in your browser you should see your Flask application's root page:
<img src="images/tada.PNG">

**Congrats!**

**Important Note:** So far, we have not been modifying files directly on the EC2 instance. This is done for a good reason. Keep in mind that directly editing files on production systems is highly prone to human error, so it should be avoided.

Typically, you should develop your Flask app locally. Test it using the local server. Then, once it works locally you can copy the files directly to EC2. 

To make this process easier, many companies automatically deploy code to EC2 each time it is committed to Github by mirroring the repo there, which is what we've done here. Alternatively, you could write a bash script that automatically copies your files to an EC2 instance. For example, Heroku offers a script to do this automatically. 

<a id="independent-practice"></a>
## Independent Practice
---

Modify the code so that it accepts another input.
You'll need to do all of the following to make it work:
- Change the input form page to add another field
- Change the myapp.py Flask code to accept the new field
- Change the Jinja template file to react to this new variable
<br><br>
**Bonus**
- Try setting it up so that your webpage uses ajax (asynchronous Javascript). Ajax is a method that makes HTTP calls to your Flask app in the background, so that loading a new page isn't necessary. You'll need to learn about jQuery to use this. See the [Additional Resources](#additional-resources) below.

<a id="conclusion"></a>
## Conclusion
---

In this lesson we have learned how to launch an AWS EC2 box from an existing AMI. We learned how to install an Apache webserver and to use WSGI to run our flask application as a daemon.

Some additional things we could have done:

- Change the MIME-type on our form to pass in a csv
- Manipulate the variables in Python/Flask before returning them
- Use a charting library to visualize what we pass to the templates
- Store and retrieve the values in a database (*Hint: try using an interface such as `SQLAlchemy`*)

<a id="additional-resources"></a>
## Additional Resources
---
- [Apache and Flask WSGI reference](https://www.jakowicz.com/flask-apache-wsgi/)
- [Apache Web Server Tutorial](https://www.digitalocean.com/community/tutorials/how-to-configure-the-apache-web-server-on-an-ubuntu-or-debian-vps)
- [Flask Docs](http://flask.pocoo.org/docs/0.11/)
- [Jinja2 Docs](http://jinja.pocoo.org/docs/dev/)
- [AJAX/jQuery](http://flask.pocoo.org/docs/0.11/patterns/jquery/)